In [21]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import regex as re
import math
from collections import defaultdict
import pytz

from rapidfuzz.fuzz import ratio

things to look at: how accurate is the prematch probability when it ends (if a team has over 50% chance to win, how often do they actually win the game?)

if my fair probability is wildely different than the midprice, should I weigh it less or find a new fair probability? How often is my direction right?

can i still find edge in a very wide market?

In [50]:
date = '2025-11-28'
odds_sport = 'cbb' #cbb, cfb, nba, nfl
kalshi_sport = 'ncaab' #ncaab, ncaaf, nba, nfl

In [ ]:
odds_df = pd.read_csv(f"../data_collection/updated_scripts/oddsapi_outputs/{date}/{odds_sport}_odds.csv")
odds_df.drop(columns=['league'], inplace=True)
odds_df.rename(columns={'price': 'odds'}, inplace=True)
odds_df['vig_prob'] = 1 / odds_df['odds']

def remove_vig_probs(df):
    df = df.copy()
    df['fair_prb'] = np.nan

    grouped = df.groupby(['game_id', 'bookmaker', 'market'])

    for _, group in grouped:
        if len(group) < 2:
            continue
        probs = group['vig_prob']
        total = probs.sum()
        if total == 0:
            continue
        fair_probs = (probs / total).round(4)
        df.loc[group.index, 'fair_prb'] = fair_probs

    return df

odds_df = remove_vig_probs(odds_df)


odds_winners_df = odds_df[odds_df['market'] == 'h2h'].copy()
odds_spreads_df = odds_df[odds_df['market'] == 'spreads'].copy()
odds_spreads_df = odds_spreads_df.loc[(odds_spreads_df['point'].notna()) & (odds_spreads_df['point'] > 0)]
odds_totals_df  = odds_df[odds_df['market'] == 'totals'].copy()

# Average per-team fair probabilities across DraftKings/FanDuel/Pinnacle for winners_df
mask = odds_winners_df['fair_prb'].notna()
avg_by_team = (
    odds_winners_df.loc[mask]
    .groupby(['game_id', 'team'])['fair_prb']
    .transform('median')
    .round(4)
)
odds_winners_df.loc[mask, 'avg_fair_prb'] = avg_by_team
odds_winners_df.loc[~mask, 'avg_fair_prb'] = pd.NA

#Average fair probabilities for spreads for same game, point spread, and team
mask = odds_spreads_df['fair_prb'].notna()
avg_by_point = (
    odds_spreads_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_spreads_df['avg_fair_prb'] = avg_by_point

#Average fair probabilities for totals for same game, point spread, direction (Over/Under)
mask = odds_totals_df['fair_prb'].notna()
avg_by_tot_point = (
    odds_totals_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform('mean')
    .round(4)
)
odds_totals_df['avg_fair_prb'] = avg_by_tot_point

In [ ]:
kalshi_winners_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_winners.csv")
kalshi_totals_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_totals.csv")
kalshi_spreads_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_spreads.csv")

if kalshi_sport == 'ncaaf':
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)
elif (kalshi_sport == 'ncaab') | (kalshi_sport == 'nba'):
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) Points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)
    



columns_to_drop = ['timestamp', 'market_type', 'yes_bid2', 'yes_ask2', 'no_bid2', 'no_ask2', 'yes_depth_bids', 'yes_depth_asks', 'no_depth_bids', 'no_depth_asks']
kalshi_winners_df.drop(columns=columns_to_drop, inplace=True)
kalshi_spreads_df.drop(columns=columns_to_drop, inplace=True)
kalshi_totals_df.drop(columns=columns_to_drop, inplace=True)

In [53]:
odds_totals_df.head()

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
4,CBB,7d14f221072dcad97c00896eb4fb91b8,2025-11-28 09:30:39 CST,FanDuel,totals,Over,1.91,152.5,Virginia Tech Hokies,VCU Rams,0.52356,0.5,0.5
5,CBB,7d14f221072dcad97c00896eb4fb91b8,2025-11-28 09:30:39 CST,FanDuel,totals,Under,1.91,152.5,Virginia Tech Hokies,VCU Rams,0.52356,0.5,0.5
10,CBB,7d14f221072dcad97c00896eb4fb91b8,2025-11-28 09:30:39 CST,DraftKings,totals,Over,1.91,152.5,Virginia Tech Hokies,VCU Rams,0.52356,0.5,0.5
11,CBB,7d14f221072dcad97c00896eb4fb91b8,2025-11-28 09:30:39 CST,DraftKings,totals,Under,1.91,152.5,Virginia Tech Hokies,VCU Rams,0.52356,0.5,0.5
16,CBB,7d14f221072dcad97c00896eb4fb91b8,2025-11-28 09:30:39 CST,ESPN BET,totals,Over,1.91,152.5,Virginia Tech Hokies,VCU Rams,0.52356,0.5,0.5


In [54]:
#get names from kalshi_winners_df
def extract_teams_from_winners(title):
    title = title.replace(" Winner?", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
    elif " vs " in title:
        right, left = title.split(" vs ", 1)
    else:
        return pd.Series([None, None])  
    left = re.sub(r'\bSt\.$', 'St', left.strip())
    right = re.sub(r'\bSt\.$', 'St', right.strip())
    return pd.Series([left, right])

kalshi_winners_df[['home_team', 'away_team']] = kalshi_winners_df['title'].apply(extract_teams_from_winners)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_winners_teams = flat_teams.tolist()

#get names from kalshi_totals_df
def extract_teams_from_totals(title):
    title = title.replace(": Total Points", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
        left = re.sub(r'\bSt\.$', 'St', left.strip())
        right = re.sub(r'\bSt\.$', 'St', right.strip())
        return pd.Series([left, right])
    return None

kalshi_totals_df[['home_team', 'away_team']] = kalshi_totals_df['title'].apply(extract_teams_from_totals)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_totals_teams = flat_teams.tolist()

#get names from kalshi_spreads_df
def extract_team_from_spreads(title):
    if " wins by " in title:
        team = title.split(" wins by ", 1)[0].strip()
        team = re.sub(r'\bSt\.$', 'St', team)
        return team
    return None

kalshi_spreads_df['team'] = kalshi_spreads_df['title'].apply(extract_team_from_spreads)
unique_teams_spread = kalshi_spreads_df['team'].drop_duplicates()
kalshi_spreads_teams = unique_teams_spread.tolist()

In [55]:
odds_teams_by_market = odds_df.groupby('market')['team'].unique().to_dict()

def fuzzy_match_kalshi_to_odds(kalshi_teams, odds_team_names):
    matched_kalshi = []
    matched_odds = []
    candidates_dict = defaultdict(list)

    kalshi_sorted = sorted(kalshi_teams, key=lambda x: x[0] if x else '')
    remaining_odds = sorted(odds_team_names.tolist().copy())

    for kalshi_name in kalshi_sorted:
        candidates = []
        for odds_name in remaining_odds:
            if kalshi_name in odds_name:
                candidates.append(odds_name)
        if len(candidates) == 1:
            candidates_dict[candidates[0]].append(kalshi_name)
        elif len(candidates) > 1:
            best_fit = candidates[0]
            best_ratio = ratio(best_fit, kalshi_name)
            for name in candidates:
                curr_ratio = ratio(name, kalshi_name)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
            candidates_dict[best_fit].append(kalshi_name)
    
    for odd, kalsh in candidates_dict.items():
        best_fit = kalsh[0]
        best_ratio = ratio(best_fit, odd)
        if len(kalsh) > 1:
            for name in kalsh:
                curr_ratio = ratio(name, odd)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
        matched_odds.append(odd)
        matched_kalshi.append(best_fit)


    return matched_kalshi, matched_odds


# Winners / h2h
matched_kalshi_h2h, matched_odds_h2h = fuzzy_match_kalshi_to_odds(
    kalshi_winners_teams,
    odds_teams_by_market.get('h2h', [])
)

# Spreads
matched_kalshi_spreads, matched_odds_spreads = fuzzy_match_kalshi_to_odds(
    kalshi_spreads_teams,
    odds_teams_by_market.get('spreads', [])
)

# Totals (match only Over/Under)
totals_odds_df = odds_df[odds_df['market'] == 'totals']
odds_totals_teams = pd.unique(totals_odds_df[['home_team', 'away_team']].values.ravel())
matched_kalshi_totals, matched_odds_totals = fuzzy_match_kalshi_to_odds(
    kalshi_totals_teams,
    odds_totals_teams
)

matched_names = {
    'h2h': {
        'kalshi': matched_kalshi_h2h,
        'odds': matched_odds_h2h
    },
    'spreads': {
        'kalshi': matched_kalshi_spreads,
        'odds': matched_odds_spreads
    },
    'totals': {
        'kalshi': matched_kalshi_totals,
        'odds': matched_odds_totals
    }
}


In [56]:
assert(len(matched_names['h2h']['kalshi']) == len(matched_names['h2h']['odds']))
assert(len(matched_names['spreads']['kalshi']) == len(matched_names['spreads']['odds']))
assert(len(matched_names['totals']['kalshi']) == len(matched_names['totals']['odds']))

In [57]:
odds_winners_df = odds_winners_df[
    odds_winners_df['home_team'].isin(matched_names['h2h']['odds']) |
    odds_winners_df['away_team'].isin(matched_names['h2h']['odds'])
].drop_duplicates(subset='team').sort_values(by='home_team').reset_index(drop=True)

kalshi_winners_df = kalshi_winners_df[
    kalshi_winners_df['home_team'].isin(matched_names['h2h']['kalshi']) |
    kalshi_winners_df['away_team'].isin(matched_names['h2h']['kalshi'])
].sort_values(by='home_team').reset_index(drop=True)

odds_spreads_df = odds_spreads_df[odds_spreads_df['team'].isin(matched_names['spreads']['odds'])].reset_index(drop=True)
kalshi_spreads_df = kalshi_spreads_df[kalshi_spreads_df['team'].isin(matched_names['spreads']['kalshi'])].reset_index(drop=True)

odds_totals_df = odds_totals_df[
    odds_totals_df['home_team'].isin(matched_names['totals']['odds']) |
    odds_totals_df['away_team'].isin(matched_names['totals']['odds'])
].sort_values(by='home_team').reset_index(drop=True)
kalshi_totals_df = kalshi_totals_df[
    (kalshi_totals_df['home_team'].isin(matched_names['totals']['kalshi'])) | 
    (kalshi_totals_df['away_team'].isin(matched_names['totals']['kalshi']))
    ].sort_values(by='home_team').reset_index(drop=True)


In [58]:
# Concatenate winners df

# Specify the columns to extract
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'no_bid', 'no_ask', 'home_team', 'away_team']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb']

# Rename overlapping columns in odds to prevent clashes
odds_subset = odds_winners_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team'
})

kalshi_subset = kalshi_winners_df[kalshi_cols].rename(columns={
    'home_team': 'kalshi_home_team',
    'away_team': 'kalshi_away_team'
})

combined_rows = []
len_matched = len(matched_names['h2h']['kalshi'])
matched_names_h2h = matched_names['h2h']

for i in range(len_matched):
    odds_name = matched_names_h2h['odds'][i]
    kalshi_name = matched_names_h2h['kalshi'][i]

    # Find the corresponding odds row
    odds_row = odds_subset.loc[odds_subset['team'] == odds_name]
    assert len(odds_row) == 1, f"Expected one row for {odds_name}, got {len(odds_row)}"

    # Find the two matching Kalshi rows
    kalshi_rows = kalshi_subset.loc[
        (kalshi_subset['kalshi_home_team'] == kalshi_name) |
        (kalshi_subset['kalshi_away_team'] == kalshi_name)
    ]
    assert len(kalshi_rows) == 2, f"Expected two rows for {kalshi_name}, got {len(kalshi_rows)}"

    # Extract rows
    k1 = kalshi_rows.iloc[0]
    k2 = kalshi_rows.iloc[1]
    midprice1 = (k1['yes_bid'] + k1['yes_ask']) / 2
    midprice2 = (k2['yes_bid'] + k2['yes_ask']) / 2

    # Extract scalar fair probability
    prb = odds_row['avg_fair_prb'].astype(float).item()

    # Choose the row closer to the odds probability
    if ((midprice1 - prb) ** 2) < ((midprice2 - prb) ** 2):
        combined_row = pd.concat([k1, odds_row.iloc[0]])
    else:
        combined_row = pd.concat([k2, odds_row.iloc[0]])

    combined_rows.append(combined_row)

combined_winners_df = pd.DataFrame(combined_rows).sort_values(by='odds_home_team')
combined_winners_df = combined_winners_df.reset_index(drop=True)

In [59]:
EDGE = 0.02
KELLY_UPPERBOUND = 1
BANKROLL = 100.00
Q1_WEIGHT = 1.00
Q2_WEIGHT = 1.00
Q3_WEIGHT = 1.00
Q4_WEIGHT = 1.00

edge_winners_df = combined_winners_df.loc[
    (combined_winners_df['avg_fair_prb'] >= combined_winners_df['yes_bid'] + EDGE) |
    (combined_winners_df['avg_fair_prb'] <= combined_winners_df['yes_ask'] - EDGE)
].reset_index(drop=True)

midprice = (edge_winners_df['yes_bid'] + edge_winners_df['yes_ask']) / 2

q_yes = edge_winners_df['avg_fair_prb']
q_no = 1 - edge_winners_df['avg_fair_prb'] 

edge_winners_df['edge'] = np.where(q_yes > midprice, q_yes - edge_winners_df['yes_bid'], q_no - edge_winners_df['no_bid'])

edge_winners_df['buy_direction'] = np.where(q_yes > midprice, "yes", "no")
edge_winners_df['raw_kelly'] = np.where(q_yes > midprice, edge_winners_df['edge'] / (1 - edge_winners_df['yes_bid']),
                                        edge_winners_df['edge'] / (1 - edge_winners_df['no_bid']))

total_kelly = edge_winners_df['raw_kelly'].sum() 
if total_kelly >= 1: 
    edge_winners_df['raw_kelly'] = pd.DataFrame({
        'original': edge_winners_df['raw_kelly'],
        'normalized': (edge_winners_df['raw_kelly'] / total_kelly)
    }).min(axis=1)

# Define the real_kelly logic
def scale_kelly(row):
    k = row['raw_kelly']
    p = row['avg_fair_prb']
    
    if k == 0 or pd.isna(k):
        return 0
    if 0.05 <= p < 0.25:
        return min(Q1_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.25 <= p < 0.5:
        return min(Q2_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.5 <= p < 0.75:
        return min(Q3_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.75 <= p < 0.95:
        return min(Q4_WEIGHT * k, KELLY_UPPERBOUND)
    else:
        return 0 

# Apply to the DataFrame
edge_winners_df['real_kelly'] = edge_winners_df.apply(scale_kelly, axis=1)
edge_winners_df['optimal_bet'] = edge_winners_df['real_kelly'] * BANKROLL

#fix this logic so that instead of shorting, should be buying 'No'. So probs should add back no_bid, no_ask columns.
q = edge_winners_df['avg_fair_prb']
p = midprice

num_contracts = np.where(q > p, edge_winners_df['optimal_bet'] // edge_winners_df['yes_bid'], edge_winners_df['optimal_bet'] // edge_winners_df['no_bid'])
edge_winners_df['num_contracts'] = num_contracts
trading_cost = np.where(q > p, np.ceil(100*(0.0175 * num_contracts * edge_winners_df['yes_bid'] * (1 - edge_winners_df['yes_bid']))) / 100,
                        np.ceil(100*(0.0175 * num_contracts * edge_winners_df['no_bid'] * (1 - edge_winners_df['no_bid']))) / 100)
edge_winners_df['trading_cost'] = trading_cost
profit = np.where(q > p, ((1 - edge_winners_df['yes_bid']) *  num_contracts - trading_cost), ((1 - edge_winners_df['no_bid']) *  num_contracts - trading_cost))
edge_winners_df['profit'] = profit
edge_winners_df['ev'] = np.where(q > p, (profit * q_yes - edge_winners_df['optimal_bet'] * (1 - q_yes)).round(2), 
                                 (profit * q_no - edge_winners_df['optimal_bet'] * (1 - q_no)).round(2))
filtered_winners_df = edge_winners_df.loc[edge_winners_df['ev'] > 0.10].reset_index(drop=True)

filtered_winners_df['start_time'] = pd.to_datetime(filtered_winners_df['start_time'])
filtered_winners_df['start_time'] = (
    filtered_winners_df['start_time']
    .dt.tz_localize('America/Chicago', ambiguous='NaT', nonexistent='NaT')
)
#now = datetime.now(pytz.timezone('America/Chicago'))
#filtered_winners_df = filtered_winners_df.loc[filtered_winners_df['start_time'] > now].sort_values('odds_home_team').reset_index(drop=True)

dupe_mask = filtered_winners_df['kalshi_home_team'].duplicated(keep=False)
dupes = filtered_winners_df[dupe_mask]
uniques = filtered_winners_df[~dupe_mask]
#among duplicates, keep only max EV per home team
best_dupes = dupes.loc[dupes.groupby('kalshi_home_team')['ev'].idxmax()]
filtered_winners_df = pd.concat([uniques, best_dupes], ignore_index=True)

/var/folders/dv/l04j1r_d2mgbb4ngbgfg__0r0000gn/T/ipykernel_26656/292549597.py:69: FutureWarning: Parsed string "2025-11-28 13:00:00 CST" included an un-recognized timezone "CST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  filtered_winners_df['start_time'] = pd.to_datetime(filtered_winners_df['start_time'])


In [60]:
team_cols = ['kalshi_home_team', 'kalshi_away_team',
             'odds_home_team', 'odds_away_team']

teams_df = filtered_winners_df[team_cols].copy()

filtered_winners_df = filtered_winners_df.drop(columns=team_cols).reset_index(drop=True)
filtered_winners_df[['edge', 'raw_kelly', 'real_kelly']] = filtered_winners_df[['edge', 'raw_kelly', 'real_kelly']].round(4) * 100

In [61]:
filtered_winners_df

,ticker,yes_bid,yes_ask,no_bid,no_ask,market,start_time,team,avg_fair_prb,edge,buy_direction,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev
0,KXNCAAMBGAME-25NOV28WRSTBUT-BUT,0.92,0.93,0.07,0.08,h2h,2025-11-28 13:00:00-06:00,Butler Bulldogs,0.9083,2.17,no,2.33,2.33,2.333333,33.0,0.04,30.65,0.69
1,KXNCAAMBGAME-25NOV28PROVFLA-FLA,0.85,0.87,0.13,0.15,h2h,2025-11-28 14:00:44-06:00,Florida Gators,0.8427,2.73,no,3.14,3.14,3.137931,24.0,0.05,20.83,0.63
2,KXNCAAMBGAME-25NOV28DEPGT-GT,0.50,0.51,0.49,0.50,h2h,2025-11-28 18:00:50-06:00,Georgia Tech Yellow Jackets,0.4778,3.22,no,6.31,6.31,6.313725,12.0,0.06,6.06,0.15
3,KXNCAAMBGAME-25NOV28STONLMU-STON,0.16,0.18,0.82,0.84,h2h,2025-11-28 21:00:00-06:00,Stony Brook Seawolves,0.1822,2.22,yes,2.64,2.64,2.642857,16.0,0.04,13.40,0.28
4,KXNCAAMBGAME-25NOV28OSUPITT-OSU,0.68,0.69,0.31,0.32,h2h,2025-11-28 18:00:00-06:00,Ohio State Buckeyes,0.6643,2.57,no,3.72,3.72,3.724638,12.0,0.05,8.23,0.29
5,KXNCAAMBGAME-25NOV28BYUDAY-DAY,0.15,0.18,0.82,0.85,h2h,2025-11-28 20:30:00-06:00,Dayton Flyers,0.1778,2.78,yes,3.27,3.27,3.270588,21.0,0.05,17.80,0.48
6,KXNCAAMBGAME-25NOV28MINNSCU-SCU,0.69,0.70,0.30,0.31,h2h,2025-11-28 20:30:00-06:00,Santa Clara Broncos,0.6743,2.57,no,3.67,3.67,3.671429,12.0,0.05,8.35,0.24
7,KXNCAAMBGAME-25NOV28NEWAKE-NE,0.05,0.07,0.93,0.95,h2h,2025-11-28 11:00:00-06:00,Northeastern Huskies,0.0710,2.10,yes,2.21,2.21,2.210526,44.0,0.04,41.76,0.91


In [46]:
total_loss = np.sum(filtered_winners_df['optimal_bet'])
total_profit = np.sum(filtered_winners_df['profit'])
total_ev = np.sum(filtered_winners_df['ev'])
print(f"{odds_sport} h2h portfolio summary:\n")
print(f"Max Loss: -{total_loss:.2f}")
print(f"Max Profit: {total_profit:.2f}")
print(f"Portfolio EV: {total_ev:.2f}")


cfb h2h portfolio summary:

Max Loss: -28.31
Max Profit: 164.64
Portfolio EV: 5.44


In [37]:
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'team', 'points']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb', 'point']

odds_subset = odds_spreads_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team',
    'team': 'odds_team'
})

kalshi_subset = kalshi_spreads_df[kalshi_cols]

combined_rows = []

for _, kalshi_row in kalshi_subset.iterrows():
    kalshi_home = kalshi_row['team']
    for _, odds_row in odds_subset.iterrows():
        odds_home = odds_row['odds_team']
        if (kalshi_home in odds_home) and (kalshi_row['points'] == odds_row['point']): 
            combined_row = pd.concat([kalshi_row, odds_row])
            combined_rows.append(combined_row)

combined_spreads_df = pd.DataFrame(combined_rows).drop_duplicates(subset='ticker') #only works because oddsapi only pulls odds 
combined_spreads_df = combined_spreads_df.reset_index(drop=True)                    #for only one point line for each bookmaker 



In [38]:
EDGE = 0.01
KELLY_UPPERBOUND = 1
BANKROLL = 25.00
Q1_WEIGHT = 1.00
Q2_WEIGHT = 1.00
Q3_WEIGHT = 1.00
Q4_WEIGHT = 1.00

edge_spreads_df = combined_spreads_df.loc[(combined_spreads_df['avg_fair_prb'] >= combined_spreads_df['yes_ask'] + EDGE) |
                        (combined_spreads_df['avg_fair_prb'] <= combined_spreads_df['yes_bid'] - EDGE)].reset_index(drop=True)

midprice = (edge_spreads_df['yes_bid'] + edge_spreads_df['yes_ask']) / 2

q = edge_spreads_df['avg_fair_prb']
p = midprice

edge_spreads_df['raw_kelly'] = np.where(
    q > p,
    (q - p) / (1 - p),      # long position
    (p - q) / p           # short position
)

total_kelly = edge_spreads_df['raw_kelly'].sum()
edge_spreads_df['raw_kelly'] = pd.DataFrame({
    'original': edge_spreads_df['raw_kelly'],
    'normalized': (edge_spreads_df['raw_kelly'] / total_kelly)
}).min(axis=1)

# Define the real_kelly logic
def scale_kelly(row):
    k = row['raw_kelly']
    p = row['avg_fair_prb']
    
    if k == 0 or pd.isna(k):
        return 0
    if 0.1 <= p < 0.25:
        return min(Q1_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.25 <= p < 0.5:
        return min(Q2_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.5 <= p < 0.75:
        return min(Q3_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.75 <= p < 0.9:
        return min(Q4_WEIGHT * k, KELLY_UPPERBOUND)
    else:
        return 0  # fallback if out of range

# Apply to the DataFrame
edge_spreads_df['real_kelly'] = edge_spreads_df.apply(scale_kelly, axis=1)
edge_spreads_df['optimal_bet'] = edge_spreads_df['real_kelly'] * BANKROLL

num_contracts = edge_spreads_df['optimal_bet'] // edge_spreads_df['yes_bid']
edge_spreads_df['num_contracts'] = num_contracts
trading_cost = np.ceil(100 * (0.0175 * num_contracts * edge_spreads_df['yes_bid'] * (1 - edge_spreads_df['yes_bid']))) / 100
edge_spreads_df['trading_cost'] = trading_cost
profit = (1 - edge_spreads_df['yes_bid']) *  num_contracts - trading_cost
edge_spreads_df['profit'] = profit
edge_spreads_df['ev'] = (profit * edge_spreads_df['avg_fair_prb'] - edge_spreads_df['optimal_bet'] * (1 - edge_spreads_df['avg_fair_prb'])).round(2)

filtered_spreads_df = edge_spreads_df.loc[edge_spreads_df['ev'] > 0].reset_index(drop=True)

filtered_spreads_df['start_time'] = pd.to_datetime(filtered_spreads_df['start_time'])
filtered_spreads_df['start_time'] = (
    filtered_spreads_df['start_time']
    .dt.tz_localize('America/Chicago', ambiguous='NaT', nonexistent='NaT')
)
now = datetime.now(pytz.timezone('America/Chicago'))
filtered_spreads_df = filtered_spreads_df.loc[filtered_spreads_df['start_time'] > now].sort_values('odds_home_team').reset_index(drop=True)


/var/folders/dv/l04j1r_d2mgbb4ngbgfg__0r0000gn/T/ipykernel_26656/1797254687.py:61: FutureWarning: Parsed string "2025-11-28 15:00:00 CST" included an un-recognized timezone "CST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  filtered_spreads_df['start_time'] = pd.to_datetime(filtered_spreads_df['start_time'])


In [39]:
edge_spreads_df

,ticker,yes_bid,yes_ask,team,points,market,start_time,odds_team,odds_home_team,odds_away_team,avg_fair_prb,point,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev
0,KXNCAAFSPREAD-25NOV28BSUUSU-USU3,0.31,0.36,Utah St,3.5,spreads,2025-11-28 15:00:00 CST,Utah State Aggies,Utah State Aggies,Boise State Broncos,0.5325,3.5,0.104403,0.104403,2.610071,8.0,0.03,5.49,1.70
1,KXNCAAFSPREAD-25NOV28BSUUSU-USU2,0.38,0.39,Utah St,2.5,spreads,2025-11-28 15:00:00 CST,Utah State Aggies,Utah State Aggies,Boise State Broncos,0.4764,2.5,0.052244,0.052244,1.306104,3.0,0.02,1.84,0.19
2,KXNCAAFSPREAD-25NOV28AFACSU-CSU2,0.37,0.40,Colorado St,2.5,spreads,2025-11-28 14:00:00 CST,Colorado State Rams,Colorado State Rams,Air Force Falcons,0.4961,2.5,0.063505,0.063505,1.587617,4.0,0.02,2.50,0.44
3,KXNCAAFSPREAD-25NOV28UTAHKU-UTAH3,0.70,0.75,Utah,3.5,spreads,2025-11-28 15:00:00 CST,Utah State Aggies,Utah State Aggies,Boise State Broncos,0.5325,3.5,0.093338,0.093338,2.333455,3.0,0.02,0.88,-0.62
4,KXNCAAFSPREAD-25NOV28UTAHKU-UTAH2,0.75,0.79,Utah,2.5,spreads,2025-11-28 15:00:00 CST,Utah State Aggies,Utah State Aggies,Boise State Broncos,0.4764,2.5,0.134039,0.134039,3.350982,4.0,0.02,0.98,-1.29
5,KXNCAAFSPREAD-25NOV28OHIOBUFF-BUFF7,0.09,0.19,Buffalo,7.5,spreads,2025-11-28 11:00:00 CST,Buffalo Bulls,Buffalo Bulls,Ohio Bobcats,0.5052,7.5,0.149279,0.149279,3.731978,41.0,0.06,37.25,16.97
6,KXNCAAFSPREAD-25NOV28UGAGT-UGA14,0.47,0.48,Georgia,14.5,spreads,2025-11-28 14:30:00 CST,Georgia Tech Yellow Jackets,Georgia Tech Yellow Jackets,Georgia Bulldogs,0.5197,14.5,0.029931,0.029931,0.748264,1.0,0.01,0.52,-0.09
7,KXNCAAFSPREAD-25NOV28UGAGT-UGA13,0.53,0.54,Georgia,13.5,spreads,2025-11-28 14:30:00 CST,Georgia Tech Yellow Jackets,Georgia Tech Yellow Jackets,Georgia Bulldogs,0.4895,13.5,0.029897,0.029897,0.747419,1.0,0.01,0.46,-0.16
8,KXNCAAFSPREAD-25NOV28TXAMTEX-TEX2,0.40,0.41,Texas,2.5,spreads,2025-11-28 18:30:00 CST,Texas Longhorns,Texas Longhorns,Texas A&M Aggies,0.4802,2.5,0.044429,0.044429,1.110728,2.0,0.01,1.19,-0.01
9,KXNCAAFSPREAD-25NOV28MISSMSST-MSST7,0.09,0.17,Mississippi St,7.5,spreads,2025-11-28 10:58:00 CST,Mississippi State Bulldogs,Mississippi State Bulldogs,Ole Miss Rebels,0.5229,7.5,0.158756,0.158756,3.968895,44.0,0.07,39.97,19.01


In [949]:
# First compute midprice in combined_winners_df
combined_winners_df['midprice'] = (
    combined_winners_df['yes_bid'] + combined_winners_df['yes_ask']
) / 2

# Merge spreads df with winner probabilities
merged = filtered_spreads_df.merge(
    combined_winners_df[['team', 'midprice']],
    left_on='odds_team',      # column in filtered_spreads_df
    right_on='team',          # column in combined_winners_df
    how='left'
)

# Filter out teams with win prob < 50% OR missing (NaN)
filtered_spreads_df = merged.loc[
    merged['midprice'] >= 0.50
].reset_index(drop=True)


In [950]:
filtered_spreads_df

,ticker,yes_bid,yes_ask,team_x,points,market,start_time,odds_team,odds_home_team,odds_away_team,...,point,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev,team_y,midprice


In [951]:
total_loss = np.sum(filtered_spreads_df['optimal_bet'])
total_profit = np.sum(filtered_spreads_df['profit'])
total_ev = np.sum(filtered_spreads_df['ev'])
print(f"{odds_sport} spreads portfolio summary:\n")
print(f"Max Loss: -{total_loss:.2f}")
print(f"Max Profit: {total_profit:.2f}")
print(f"Portfolio EV: {total_ev:.2f}")


cfb spreads portfolio summary:

Max Loss: -0.00
Max Profit: 0.00
Portfolio EV: 0.00
